In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from heapq import nlargest
from nltk.tokenize import sent_tokenize  # Bạn cần import thư viện nltk để sử dụng hàm sent_tokenize

def generate_summary(text, n):
    # Tách văn bản thành các câu
    sentences = sent_tokenize(text)

    # Tạo ma trận TF-IDF
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Tính độ tương đồng cosine giữa mỗi câu và văn bản
    sentence_scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]

    # Chọn n câu có điểm số cao nhất
    summary_sentences = nlargest(n, range(len(sentence_scores)), key=sentence_scores.__getitem__)

    # Tạo tóm tắt từ các câu được chọn
    summary_tfidf = ' '.join([sentences[i] for i in sorted(summary_sentences)])

    return summary_tfidf


In [4]:
text = '''
By . James Slack, Home Affairs Editor . PUBLISHED: . 19:55 EST, 6 January 2013 . | . UPDATED: . 03:39 EST, 7 January 2013 .
Cameron wants a policy that allows UK to deport terror suspects before they start appeal procedure .
Terror suspects could face a new system of being deported before they get the chance to hold a full appeal, David Cameron revealed yesterday. 
The Prime Minister said he was fed up with the likes of hate-preacher Abu Qatada using a string of human rights appeals to remain in the UK.
Mr Cameron went on: Thats why Im keen to move to a policy where we deport first, and suspects can appeal later.
Under this new arrangement, deportees would only be able to appeal against the decision while still in this country - suspending their removal - if they faced a real risk of serious, irreversible harm. Cur
rently, nobody can be put on a plane until all their appeals to British courts - and Strasbourg - have been exhausted. The proposals go further than ministers have ever previously suggested. 
But last night government officials were unable to provide further details of how the new regime would operate. It was also unclear whether it would apply only to suspected terrorists. 
Currently, there are 4,000 foreign rapists, muggers and other criminals walking the streets who the UK cannot kick out. New rules limited only to the extradition of terror suspects would fall far short of the demands made by Tory MPs for a wide-ranging overhaul of human rights law.
There are also question marks over whether Mr Camerons idea would be acceptable to either the Liberal Democrats or, crucially, the European Court of Human Rights. Ministers would have to explain how, in the event of a terror suspect winning their appeal from overseas, they would be able to locate the terror suspect and bring them back to Britain.
Abu Qatada has used a string of human rights appeals to remain in the UK at the taxpayer's expense . Theresa May wants to reduce 'the unacceptable delays' in the extradition process . Strasbourg has taken a hard-line on removing anybody to a country where torture or ill-treatment takes place. Qatada’s deportation to Jordan was blocked by Strasbourg on the grounds not that he would be harmed himself, but that some of the evidence used against him may have been obtained by torture.
 He is currently living in a £400,000 London house - paid for by the taxpayer - pending an appeal by Home Secretary Theresa May. Last night Tory MP Dominic Raab, who has campaigned to stop the abuse of human rights law by foreign criminals, said: ‘The Prime Minister is absolutely right to embrace fundamental deportation reform. ‘For it to work, we will need an Act of Parliament that lays down the law in clear terms, so ingenious judges with other agendas - at home and in Strasbourg - can’t subvert the common sense system the British public want put in place.’ In October last year, Home Secretary Theresa May said there was a need to reduce ‘the wholly unacceptable delays’ that have occurred in the extradition process for terror suspects. She said there was ‘scope for reforming rights of appeal, streamlining the stages, expediting cases through the court and looking again at the provision of legal aid for terrorist suspects’. Yesterday, Mr Cameron also voiced frustration with the pace of change on immigration reform, which he said was ‘just an issue where there is quite a profound disagreement between the Liberal Democrats and Conservatives
.The Prime Minister said it is  an issue of the absolute centre ground of British politics'''

summary = generate_summary(text, 3)
summary_sentences = summary.split('. ')
formatted_summary = '.\n'.join(summary_sentences)

print(formatted_summary)

The Prime Minister said he was fed up with the likes of hate-preacher Abu Qatada using a string of human rights appeals to remain in the UK.
There are also question marks over whether Mr Cameron’s idea would be acceptable to either the Liberal Democrats or, crucially, the European Court of Human Rights.
Last night Tory MP Dominic Raab, who has campaigned to stop the abuse of human rights law by foreign criminals, said: ‘The Prime Minister is absolutely right to embrace fundamental deportation reform.
